In [4]:
# Bibliotecas
import sys
import os
from newsapi import NewsApiClient

In [5]:
# Variáveis
sys.path.append(os.path.abspath(os.path.join(os.path.dirname("tests.ipynb"), "..", "env")))
from env_variables import apikey as apikey_temp

In [6]:
# Scripts
sys.path.append(os.path.abspath(os.path.join(os.path.dirname("tests.ipynb"), "..", "api")))
from api import funcao_api

In [7]:
news_api = funcao_api(apikey_temp)
news_api

In [15]:
all_articles = news_api.get_everything(q='genomics')

all_articles["totalResults"]

606